In [3]:
from unittest import loader
import monai
import torch

image_dir_no_patho = ["./data/ATLAS/splits_fid/atlas_no_patho_png.csv"]
image_dir_patho_sub1 = ["./data/ATLAS/splits_fid/atlas_patho_sub1_png.csv"]
image_dir_patho_sub2 = ["./data/ATLAS/splits_fid/atlas_patho_sub2_png.csv"]
image_dir_ixi = ["./data/ATLAS/splits_fid/ixi_png.csv"]

from torch.utils.data import DataLoader
import sys
import os

os.chdir('/home/malek/autoDDPM')
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))


# Get the parent directory
# Add the parent directory to the system path
sys.path.append(dir_path)
print(dir_path)
from data.loaders.ixi_loader import mask_preprocessing_loader, AtlasLoader, IXILoaderHarmonize, AtlasLoaderHarmonize

from PIL import Image
import numpy as np


device = "cuda" if torch.cuda.is_available() else "cpu"

def compute_data_loaders(mni_harmonizing):
    if mni_harmonizing:
        dataset_no_patho = AtlasLoaderHarmonize(
                    image_dir_no_patho,
                    target_size=(128, 128),
                    test=False,
                )

        loader_no_patho = DataLoader(
            dataset_no_patho, batch_size=150, shuffle=False, drop_last=False, pin_memory=True
        )

        dataset_patho_sub1 = AtlasLoaderHarmonize(
                    image_dir_patho_sub1,
                    target_size=(128, 128),
                    test=False,
                )

        loader_patho_sub1 = DataLoader(
            dataset_patho_sub1, batch_size=150, shuffle=False, drop_last=False, pin_memory=True
        )

        dataset_patho_sub2 = AtlasLoaderHarmonize(
                    image_dir_patho_sub2,
                    target_size=(128, 128),
                    test=False,
                )

        loader_patho_sub2 = DataLoader(
            dataset_patho_sub2, batch_size=150, shuffle=False, drop_last=False, pin_memory=True
        )

        dataset_ixi = IXILoaderHarmonize(
                    image_dir_ixi,
                    target_size=(128, 128),
                    test=False,
                )
    else:
        dataset_no_patho = AtlasLoader(
                    image_dir_no_patho,
                    target_size=(128, 128),
                    test=False,
                )

        loader_no_patho = DataLoader(
            dataset_no_patho, batch_size=150, shuffle=False, drop_last=False, pin_memory=True
        )

        dataset_patho_sub1 = AtlasLoader(
                    image_dir_patho_sub1,
                    target_size=(128, 128),
                    test=False,
                )

        loader_patho_sub1 = DataLoader(
            dataset_patho_sub1, batch_size=150, shuffle=False, drop_last=False, pin_memory=True
        )

        dataset_patho_sub2 = AtlasLoader(
                    image_dir_patho_sub2,
                    target_size=(128, 128),
                    test=False,
                )

        loader_patho_sub2 = DataLoader(
            dataset_patho_sub2, batch_size=150, shuffle=False, drop_last=False, pin_memory=True
        )

        dataset_ixi = AtlasLoader(
                    image_dir_ixi,
                    target_size=(128, 128),
                    test=False,
                )

    loader_ixi = DataLoader(
        dataset_ixi, batch_size=150, shuffle=False, drop_last=False, pin_memory=True
    )
    return loader_no_patho, loader_patho_sub1, loader_patho_sub2, loader_ixi


/home/malek


In [6]:
def subtract_mean(x: torch.Tensor) -> torch.Tensor:
    mean = [0.406, 0.456, 0.485]
    x[:, 0, :, :] -= mean[0]
    x[:, 1, :, :] -= mean[1]
    x[:, 2, :, :] -= mean[2]
    return x


def spatial_average(x: torch.Tensor, keepdim: bool = True) -> torch.Tensor:
    return x.mean([2, 3], keepdim=keepdim)


def get_features(image):
    # If input has just 1 channel, repeat channel to have 3 channels
    if image.shape[1]:
        image = image.repeat(1, 3, 1, 1)

    # Change order from 'RGB' to 'BGR'
    image = image[:, [2, 1, 0], ...]

    # Subtract mean used during training
    image = subtract_mean(image)

    # Get model outputs
    with torch.no_grad():
        feature_image = radnet.forward(image)
        # flattens the image spatially
        feature_image = spatial_average(feature_image, keepdim=False)

    return feature_image


In [7]:
import torch

def psnr(original, reconstructed, max_val=1):
    mse = torch.mean(torch.square(original - reconstructed))
    psnr = 10 * torch.log10((max_val ** 2) / mse)
    return psnr





In [10]:
mni_harmonizing = True
loader_no_patho, loader_patho_sub1, loader_patho_sub2, loader_ixi = compute_data_loaders(mni_harmonizing)
for (data_1,data_2,data_3,data_4) in zip(loader_no_patho,loader_patho_sub1,loader_patho_sub2,loader_ixi):
    images_no_patho = data_1[0].to(device)
    images_patho_sub1 = data_2[0].to(device)
    images_patho_sub2 = data_3[0].to(device)
    images_ixi = data_4[0].to(device)

    print(images_no_patho.shape)
    print(images_patho_sub1.shape)
    print(images_patho_sub2.shape)
    print(images_ixi.shape)



shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128, 128])
shape of image_np (128, 128)
shape of image_reference (128, 128)
shape of matched_tensor torch.Size([128

In [11]:
mni_harmonizing = False
loader_no_patho_non_harm, loader_patho_sub1_non_harm, loader_patho_sub2_non_harm, loader_ixi_non_harm = compute_data_loaders(mni_harmonizing)

for data_1,data_2,data_3,data_4 in zip(loader_no_patho_non_harm,loader_patho_sub1_non_harm,loader_patho_sub2_non_harm,loader_ixi_non_harm):
    images_no_patho_non_harm = data_1[0].to(device)
    images_patho_sub1_non_harm = data_2[0].to(device)
    images_patho_sub2_non_harm = data_3[0].to(device)
    images_ixi_non_harm = data_4[0].to(device)

    print(images_no_patho_non_harm.shape)
    print(images_patho_sub1_non_harm.shape)
    print(images_patho_sub2_non_harm.shape)
    print(images_ixi_non_harm.shape)




torch.Size([150, 1, 128, 128])
torch.Size([150, 1, 128, 128])
torch.Size([150, 1, 128, 128])
torch.Size([150, 1, 128, 128])


In [16]:
# Example usage:
# Assuming original_image and reconstructed_image are torch tensors with shape [B, 1, H, W]
psnr_value = psnr(images_no_patho, images_no_patho_non_harm)
print("PSNR non pathological atlas:", psnr_value.item())

PSNR non pathological atlas: 16.13991928100586


In [17]:
psnr_value = psnr(images_patho_sub1, images_patho_sub1_non_harm)
print("PSNR pathological atlas subset 1:", psnr_value.item())

PSNR pathological atlas subset 1: 16.175857543945312


In [18]:
psnr_value = psnr(images_patho_sub2, images_patho_sub2_non_harm)
print("PSNR pathological atlas subset 2:", psnr_value.item())


PSNR pathological atlas subset 2: 16.0672607421875


In [19]:
psnr_value = psnr(images_ixi, images_ixi_non_harm)
print("PSNR IXI :", psnr_value.item())

PSNR IXI : 27.195837020874023
